In [2]:
!pip install tensorflow==2.16.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 31.7 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Fo

In [3]:
import tensorflow as tf
tf.__version__

'2.16.1'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Since the model size is large we cannot upload it on github. If you want to access the model, send request to adilaqzma@gmail.com and give path to the model.
#OR Link for access: https://drive.google.com/drive/folders/1DtNLKAVUmHkDrXgHj-CIuYqCBpdBHgZ3?usp=sharing
model_path = '/content/drive/MyDrive/Deepfake Project/xception_deepfake_image.h5'

In [6]:
!pip install pyngrok
!pip install flask_ngrok

In [7]:
from flask import Flask
from pyngrok import ngrok

In [8]:
import numpy as np
import cv2
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import os
from pyngrok import ngrok  # Import ngrok

In [9]:
port_no = 5000

In [ ]:
  from flask import Flask, render_template, request, redirect, url_for
  from werkzeug.utils import secure_filename
  import os
  from pyngrok import ngrok

  app = Flask(__name__, template_folder='/content/drive/MyDrive/Deepfake Project/Flask_UI/templates', static_folder='/content/drive/MyDrive/Deepfake Project/Flask_UI/static')

  # app = Flask(__name__, template_folder='/content/drive/MyDrive/Deepfake Project/Flask_UI/templates')
  # app = Flask(__name__, static_url_path='/content/drive/MyDrive/Deepfake Project/Flask_UI/static')

  ngrok.set_auth_token("2fgbipMOSmvcRrqLlOrKIknzGp9_33UT34n36ZJJXfsPVsiCR")
  public_url =  ngrok.connect(port_no).public_url

  # Define upload folder and allowed extensions
  UPLOAD_FOLDER = '/content/drive/MyDrive/Deepfake Project/Flask_UI/static/uploads'
  ALLOWED_EXTENSIONS = {'jpg', 'jpeg', 'png'}
  app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

  model = load_model('/content/drive/MyDrive/Deepfake Project/xception_deepfake_image.h5')

  # Function to check if the file extension is allowed
  def allowed_file(filename):
      return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

  # Function to preprocess the image
  def preprocess_image(image):
      input_size = (224, 224)
      resized_image = cv2.resize(image, input_size)
      normalized_image = resized_image / 255.0
      return np.expand_dims(normalized_image, axis=0)

  # Function to make predictions
  def make_prediction(image):
      preprocessed_image = preprocess_image(image)
      predictions = model.predict(preprocessed_image)
      prediction_value = predictions[0][0]
      return prediction_value

  # Route for home page
  @app.route('/')
  def home():
      return render_template('index.html')

  # Route to handle image upload and prediction
  @app.route('/upload', methods=['POST'])
  def upload_file():
      if request.method == 'POST':
          if 'file' not in request.files:
              return redirect(request.url)
          file = request.files['file']
          print("###############",file.filename)
          if file.filename == '':
              return redirect(request.url)
          if file and allowed_file(file.filename):
              filename = secure_filename(file.filename)
              # Check if the upload folder exists, if not create it
              if not os.path.exists(UPLOAD_FOLDER):
                  os.makedirs(UPLOAD_FOLDER)
              filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
              print("AAAAAAAAAAAA",filepath)
              file.save(filepath)
              image = cv2.imread(filepath)
              prediction_value = make_prediction(image)
              if prediction_value >= 0.5:
                  classification_label = "DEEPFAKE"
                  confidence = 1 - prediction_value
              else:
                  classification_label = "REAL"
                  confidence = 1 - prediction_value
              # file = os.path.join(filepath, file.filename)
              # return render_template('result.html', filename=filename, classification=classification_label, confidence=confidence)
              image_name = file.filename
              # image_name=image_name

              return render_template('result.html', classification = classification_label, confidence = confidence,image_name=image_name)
              # return render_template('newPage.html',classification= classification_label, confidence=confidence)

  print(f"To acces the Gloable link please click {public_url}")

  app.run(port=port_no)

To acces the Gloable link please click https://320b-34-91-117-26.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
